In [64]:
import pandas as pd
import json
import os
from tqdm import tqdm

In [81]:
with open('/opt/ml/input/dialogue_data/Training/KAKAO/KAKAO_1000_01.json','r') as f:
    temp = json.load(f)

In [82]:
temp.keys()

dict_keys(['dataset', 'licenses', 'info'])

In [83]:
temp['dataset']

{'identifier': 71459,
 'name': 'KAKAO_1000_01_set',
 'src_path': '/data/file/cubeManager/PROJECT001/53/txt20211007113435043254/KAKAO_1000_01_set/',
 'label_path': '/data/file/cubeManager/PROJECT001/53/txt20211007113435043254/KAKAO_1000_01_set/',
 'category': 2,
 'type': 0}

In [84]:
temp['licenses']

{'name': 'Apache License 1.0',
 'url': 'http://www.apache.org/licenses/LICENSE-1.0'}

In [85]:
temp['info']

[{'id': 47575,
  'filename': 'KAKAO_1000_01.txt',
  'title': 'KAKAO_1000_01',
  'mediatype': 'SNS',
  'medianame': '카카오톡',
  'category': '일상대화',
  'date': '2021-10-07',
  'size': 739,
  'annotations': {'subject': '식음료',
   'speaker_type': '다자간 대화',
   'size': 739,
   'word_size': 242,
   'text': '1 : 애덜앙 나 너무 배불러서 배 아파\n2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때\n3 : 얼마나 먹었다고 배가 아프데\n1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어\n2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지\n3 : 아닌데 그냥 많이 먹은 거 같은데\n1 : 그냥 나 뭐 사주기 싫어서 그런 거 같은데.\n2 : 겨우 떡볶이랑 치킨이랑 튀김이랑 닭껍질튀김 가지고 그러면 어떡해 그마저도 내가 다 먹었는데\n3 : 스테키 샐러드 마렵다 ㅠ 다이어트 겸 ㅎ\n1 : 샐러드는 빼고 스테이크만 먹고 싶은데 그것도 다이어트 될까?\n2 : 샐러드 진짜 하루 종일 먹어줄 수 있는데\n3 : 샐러드는 하루 종일 먹어도 배고파\n1 : 소도 풀만 먹는다. 샐러드라고 많이 먹어도 되는 건 아니야\n2 : 그니까 하루 종일 먹을 수 있는 거지 하루 종일 배고프니까\n3 : 요즘은 샐러드 엄청 비싸게 팔더라\n1 : 샐러드? 그냥 산 가서 쑥 같은 거 좀 캐 가지고 드레싱 뿌리면 되는 거 아님?\n2 : 풀때기 몇 개에 계란 하나 올려준 거 쿠팡에서 많이 나가긴 하던데\n3 : 키키 샐러드 파는 가게 많잖아 거의 7천원 대야\n1 : 도대체 샐러드를 왜 돈 주고 사 먹는 거야?\n1 : 난 풀 취급 안 한다\n2 : 샐러드 두 개 먹을 돈이면 치킨 한 마리 시켜 먹는다\n3 : 너가 생각하는 그런 풀이 

In [74]:
def dialogue_sort(example):
    ex_dict = {}
    speech_Act = []
    speech_Cat = []
    norm_texts = []
    morph_list = []
    # shortened variables
    infos = example['info'][0]
    annots = infos['annotations']
    # common properties
    ex_dict['id'] = infos['id']
    ex_dict['media'] = infos['medianame']
    ex_dict['date'] = infos['date']
    ex_dict['category'] = infos['category']
    # annotation properties
    ex_dict['subject'] = annots['subject']
    ex_dict['speaker_type'] = annots['speaker_type']
    ex_dict['raw_text'] = annots['text']
    ex_dict['raw_lines'] = annots['lines']
    # line properties
    for i in range(len(annots['lines'])):
        # 화행 정보 분류
        speech_Act.append(annots['lines'][i]['speechAct'])
        # 0 : 미분류, 1: 지시, 2: 단언, 5: 언약, 6: 표현
        if annots['lines'][i]['speechAct'][:4] == '(지시)':
            speech_Cat.append(2)
        elif annots['lines'][i]['speechAct'][:4] == '(단언)':
            speech_Cat.append(1)
        elif annots['lines'][i]['speechAct'][:4] == '(언약)':
            speech_Cat.append(5)
        elif annots['lines'][i]['speechAct'][:4] == '(표현)':
            speech_Cat.append(6)
        else:
            speech_Cat.append(0)
        norm_texts.append(annots['lines'][i]['norm_text'])
        morph_list.append(annots['lines'][i]['morpheme'])
    # line properties to dict
    ex_dict['norm_texts'] = norm_texts
    ex_dict['text_morphs'] = morph_list
    ex_dict['speech_Act'] = speech_Act
    ex_dict['speech_Cat'] = speech_Cat
    return ex_dict

In [90]:
train_path = "/opt/ml/input/dialogue_data/Training/Train"
valid_path = "/opt/ml/input/dialogue_data/Validation/Valid"

In [96]:
data_list = []
extra_list = []
# train path / valid path switch
for (root, directories, files) in os.walk(valid_path):
    for file in tqdm(files):
        if '.json' in file:
            file_path = os.path.join(root, file)
            with open(file_path,'r') as f:
                try:
                    temp = json.load(f)
                    data_list.append(dialogue_sort(temp))
                except:
                    extra_list.append(file_path)

100%|██████████| 10962/10962 [00:06<00:00, 1620.72it/s]


In [97]:
len(extra_list)

0

In [94]:
pd.DataFrame(data_list)

,id,media,date,category,subject,speaker_type,raw_text,raw_lines,norm_texts,text_morphs,speech_Act,speech_Cat
0,132160,카카오톡,2021-11-03,일상대화,교통,다자간 대화,1 : 버스타고 기분 좋았던 경험?\n2 : 나는 버스타고 정처없이 떠돌 때 기분 ...,"[{'id': 1, 'text': '1 : 버스타고 기분 좋았던 경험?', 'nor...","[버스타고 기분 좋았던 경험?, 나는 버스타고 정처없이 떠돌 때 기분 좋더라, 사람...",[버스/NNG+타/VV+고/EC+기분/NNG+좋/VA+았/EP+던/ETM+경험/NN...,"[(지시) 질문하기, (표현) 긍정감정 표현하기, (표현) 긍정감정 표현하기, (단...","[2, 6, 6, 1, 1, 6, 2, 1, 2, 6, 2, 6]"
1,46838,카카오톡,2021-10-06,일상대화,미용,1:1,1 : 요즘 모공관리가 고민이야.\n1 : 모공관리하는 너만의 비법이 있니?\n2 ...,"[{'id': 1, 'text': '1 : 요즘 모공관리가 고민이야.', 'norm...","[요즘 모공관리가 고민이야., 모공관리하는 너만의 비법이 있니?, 나는 가끔 스크럽...","[요즘/NNG+모공/NNG+관리/NNG+가/JKS+고민/NNG+이/VCP+야/EF,...","[(단언) 주장하기, (지시) 질문하기, (단언) 진술하기, (단언) 진술하기, (...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,133919,카카오톡,2021-11-03,일상대화,미용,다자간 대화,1 : 피부에 젤 중요한 건 선크림이다!\n2 : 맞아 키키 선크림 매일 바르고 다...,"[{'id': 1, 'text': '1 : 피부에 젤 중요한 건 선크림이다!', '...","[피부에 젤 중요한 건 선크림이다!, 맞아 키키 선크림 매일 바르고 다니지, 선크림...",[피부/NNG+에/JKB+젤/MAG+중요/NNG+한/XSA+ETM+건/NNB+JX+...,"[(단언) 진술하기, (단언) 진술하기, (단언) 진술하기, (단언) 주장하기, (...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, ..."
3,90019,카카오톡,2021-10-25,일상대화,교통,1:1,1 : 회사랑은 위치 가깝나 ?\n2 : 차 타고 30분 정도?\n1 : 아 꽤 거...,"[{'id': 1, 'text': '1 : 회사랑은 위치 가깝나 ?', 'norm_...","[회사랑은 위치 가깝나 ?, 차 타고 30분 정도?, 아 꽤 거리가 있는 편이네 ?...","[회사/NNG+랑/JKB+은/JX+위치/NNG+가깝/VA+나/EF, 차/NNG+타/...","[(지시) 질문하기, (단언) 주장하기, (단언) 진술하기, (단언) 주장하기, (...","[2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, ..."
4,81331,카카오톡,2021-10-20,일상대화,미용,1:1,1 : 요즘 미장원에 손님들이 많아요?\n2 : 코로나 초창기에는 얼마 없었는데 요...,"[{'id': 1, 'text': '1 : 요즘 미장원에 손님들이 많아요?', 'n...","[요즘 미장원에 손님들이 많아요?, 코로나 초창기에는 얼마 없었는데 요즘엔 많아졌어...",[요즘/NNG+미장원/NNG+에/JKB+손님/NNG+들/XSN+이/JKS+많/VA+...,"[(지시) 질문하기, (단언) 진술하기, (단언) 주장하기, (단언) 주장하기, (...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...
87684,105324,인스타그램,2021-10-29,일상대화,스포츠/레저,1:1,1 : 오랜만에 캠핑 가고싶다.\n2 : 캠핑? 원래 좋아했었어?\n1 : 그건 아...,"[{'id': 1, 'text': '1 : 오랜만에 캠핑 가고싶다.', 'norm_...","[오랜만에 캠핑 가고싶다., 캠핑? 원래 좋아했었어?, 그건 아닌데 유튜브에서 간 ...","[오랜만/NNG+에/JKB+캠핑/NNG+가/VV+고/EC+싶/VX+다/EF, 캠핑/...","[(단언) 주장하기, (지시) 질문하기, (단언) 진술하기, (단언) 주장하기, (...","[1, 2, 1, 1, 2, 2, 5, 5, 1, 5]"
87685,77575,카카오톡,2021-10-19,일상대화,상거래 전반,1:1,1 : 저는 지난번 말씀드린 오아시스를 엄청 애용해요\n2 : 오이시스에서 뭐 주로...,"[{'id': 1, 'text': '1 : 저는 지난번 말씀드린 오아시스를 엄청 애...","[저는 지난번 말씀드린 오아시스를 엄청 애용해요, 오이시스에서 뭐 주로 사시나요?,...",[저/NP+는/JX+지난번/NNG+말씀/NNG+드린/VV+ETM+오아시스/NNG+를...,"[(단언) 진술하기, (지시) 질문하기, (단언) 진술하기, (지시) 질문하기, (...","[1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1]"
87686,3528,카카오톡,2021-08-31,일상대화,영화/만화,1:1,1 : 야 나 영화 카고 봤었거든\n1 : 샵검색 : #넷플릭스 카고\n1 : 이...,"[{'id': 1, 'text': '1 : 야 나 영화 카고 봤었거든', 'norm...","[야 나 영화 카고 봤었거든, 샵검색 : #넷플릭스 카고, : 이거 재밌음 나중에 ...","[야/IC+나/NP+영화/NNG+카/VV+고/EC+봤었/VX+EP+거든/EC, 샵/...","[(단언) 진술하기, (단언) 진술하기, (단언) 주장하기, (지시) 질문하기, (...","[1, 1, 1, 2, 1, 1, 0, 5, 1, 1, 1, 1, 1, 1, 1, ..."
87687,167210,카카오톡,2021-11-15,일상대화,방송/연예,1:1,1 : 오늘은 구경이 봐야 되는디 하하\n2 : 구경이 그거 재밌드라 키키\n1 :...,"[{'id': 1, 'text': '1 : 오늘은 구경이 봐야 되는디 하하', 'n...","[오늘은 구경이 봐야 되는디 하하, 구경이 그거 재밌드라 키키, 앗 벌써 시작했겠네...",[오늘/NNG+은/JX+구경/NNG+이/JKS+봐야/VV+EC+되/VV+는/ETM+...,"[(단언) 주장하기, (단언) 주장하기, (단언) 주장하기, (지시) 질문하기, (...","[1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]"


In [98]:
pd.DataFrame(data_list).to_csv('/opt/ml/input/dialogue_data/valid.csv')